# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [134]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [135]:
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
os.getenv('LOG_DIR')
from logger import get_logger
_logs = get_logger(__name__)

In [136]:
#import dask
#dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [137]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(PRICE_DATA+"/**/*.parquet/*.parquet")
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [138]:
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

In [139]:
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613018,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281456,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782803,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344170,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372845,2819626,Health Care,Life Sciences Tools & Services,2000


In [140]:
# Write your code below.
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign( 
    hi_lo_range = lambda x: x["High"] - x["Low"]
#    positive_return = lambda x: (x['returns'] > 0)*1
))

C:\Users\sarit\AppData\Local\Temp\ipykernel_28908\2896824182.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [141]:
type(dd_feat)

dask.dataframe.core.DataFrame

In [142]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
npartitions=11242,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [143]:
# The compute function result is a pandas dataframe which we can use in the rolling mean calculation
# or pd.DataFrame(dd_feat)
df = dd_feat.compute()

In [144]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
OXY,2010-01-04,79.193359,80.007706,78.752647,79.614906,52.235340,3630352,Energy,Oil & Gas Exploration & Production,2010,NaN,NaN,1.255058
OXY,2010-01-05,79.691551,79.739449,78.589775,79.040070,51.858189,3515432,Energy,Oil & Gas Exploration & Production,2010,79.614906,-0.007220,1.149673
OXY,2010-01-06,78.944260,80.352608,78.398163,79.988548,52.480484,5578661,Energy,Oil & Gas Exploration & Production,2010,79.040070,0.012000,1.954445
OXY,2010-01-07,79.471191,80.122681,78.915520,79.394547,52.090763,4300559,Energy,Oil & Gas Exploration & Production,2010,79.988548,-0.007426,1.207161
OXY,2010-01-08,79.059227,80.266388,78.110748,80.208900,52.625076,4336465,Energy,Oil & Gas Exploration & Production,2010,79.394547,0.010257,2.155640


In [145]:
# Write your code below.
df = df.sort_values('Date')
df['rolling_avg_10'] = df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window =10).mean())
# display the result to see that mean rolling avg is seen  10th row onwards
_logs.info(f'rolling_avg {df.groupby("ticker")["rolling_avg_10"].head(10)}')
#_logs.info(f'rolling_avg {df.groupby("ticker")["rolling_avg_10"].head(11).tolist()}')

2024-06-29 18:29:24,924, 1728809087.py, 5, INFO, rolling_avg ticker
NKE           NaN
NVDA          NaN
CI            NaN
LHX           NaN
ALB           NaN
          ...    
VLTO          NaN
VLTO          NaN
VLTO          NaN
VLTO          NaN
VLTO    74.908001
Name: rolling_avg_10, Length: 4990, dtype: float64
2024-06-29 18:29:24,924, 1728809087.py, 5, INFO, rolling_avg ticker
NKE           NaN
NVDA          NaN
CI            NaN
LHX           NaN
ALB           NaN
          ...    
VLTO          NaN
VLTO          NaN
VLTO          NaN
VLTO          NaN
VLTO    74.908001
Name: rolling_avg_10, Length: 4990, dtype: float64
2024-06-29 18:29:24,924, 1728809087.py, 5, INFO, rolling_avg ticker
NKE           NaN
NVDA          NaN
CI            NaN
LHX           NaN
ALB           NaN
          ...    
VLTO          NaN
VLTO          NaN
VLTO          NaN
VLTO          NaN
VLTO    74.908001
Name: rolling_avg_10, Length: 4990, dtype: float64
2024-06-29 18:29:24,924, 1728809087.py, 5, INFO, 

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

--> I do not think so.A dask dataframe is  a large parallel dataframe composed of many smaller pandas dataframes split along the index.We could have used compute after calculating the rolling average.
Also,it is not necessary to explicitly convert to pandas as the compute function creates a pandas df which may be easier to do calculations on. 
+ Would it have been better to do it in Dask? Why?

-->It would have been faster to do this in dask as the computation could have been done parallely or in cases where the data was too large for memory. 

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.